## 실시간 얼굴 인식 및 표정 분석 시스템 구현

In [1]:
from dotenv import load_dotenv
import os

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")
# print(API_KEY)

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

from flask import Flask, request, jsonify, render_template
app = Flask(__name__)

### [1] 카메라를 사용한 얼굴 표정 감지

In [2]:
import cv2
import numpy as np
import base64
import time 

# 이미지를 Base64로 인코딩
def encode_image_to_base64(image):    
    _, buffer = cv2.imencode('.jpg', image)
    return base64.b64encode(buffer).decode('utf-8')

def analyze_emotion(face_image):
    # 이미지 Base64 인코딩
    image_base64 = encode_image_to_base64(face_image)

    response = client.chat.completions.create(
      model = "gpt-4o-mini",     
      messages = [
        {
          "role": "user",
          "content": [
            {"type": "text", "text": "이미지의 표정을 보고 기쁘면 [Happy],슬프면 [Sad],놀란 표정이면 [Surprised], 화났으면 [Angry],무표정은 [Neutral]으로 말해줘"},
            {
              "type": "image_url",
              "image_url": {
                 "url": f"data:image/jpeg;base64,{image_base64}"
              },
            },
          ],
        }
      ],
      
    )    
    return response.choices[0].message.content
    

# OpenCV 얼굴 감지기 로드
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 웹캠 실행
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        
        emotion = analyze_emotion(face_roi)
        # print(emotion)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    cv2.imshow('Real-time Face and Emotion Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### [2] 웹 서비스를 사용한 실시간 얼굴 표정 감지

In [3]:
from flask import Flask, Response, render_template
import cv2
import base64

# app = Flask(__name__)

# 이미지를 Base64로 인코딩
def encode_image_to_base64(image):    
    _, buffer = cv2.imencode('.jpg', image)
    return base64.b64encode(buffer).decode('utf-8')

def analyze_emotion(face_image):
    # 이미지 Base64 인코딩
    image_base64 = encode_image_to_base64(face_image)

    response = client.chat.completions.create(
      model = "gpt-4o-mini",     
      messages = [
        {
          "role": "user",
          "content": [
            {"type": "text", "text": "이미지의 표정을 보고 기쁘면 [Happy],슬프면 [Sad],놀란 표정이면 [Surprised], 화났으면 [Angry],무표정은 [Neutral]으로 말해줘"},
            {
              "type": "image_url",
              "image_url": {
                 "url": f"data:image/jpeg;base64,{image_base64}"
              },
            },
          ],
        }
      ],
      
    )    
    return response.choices[0].message.content
    

# OpenCV 얼굴 감지기 로드
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# OpenCV를 이용해 카메라 캡처 (0은 기본 웹캠)
cap = cv2.VideoCapture(0)
        
def generate_frames():
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        for (x, y, w, h) in faces:
            face_roi = frame[y:y+h, x:x+w]
            
            emotion = analyze_emotion(face_roi)
            # print(emotion)
            
            rect_img = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            text_img  = cv2.putText(rect_img, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)        

            _, buffer = cv2.imencode('.jpg', text_img)  # 프레임을 JPEG로 변환
            frame_bytes = buffer.tobytes()
            
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')  # HTML 페이지 렌더링

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.23:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Mar/2025 04:08:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 04:08:35] "GET /video_feed HTTP/1.1" 200 -


### [3] 속도 개선 - 1 : OpenCV 프레임 캡처를 별도의 쓰레드에서 실행
카메라 프레임을 메인 스레드에서 직접 읽는 방식이 아니라, 백그라운드 쓰레드에서 미리 읽어둔 프레임을 사용하면 속도가 개선 된다

In [1]:
from dotenv import load_dotenv
import os

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")
# print(API_KEY)

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

from flask import Flask, request, jsonify, render_template
app = Flask(__name__)

In [2]:
from flask import Flask, Response, render_template
import cv2
import threading
import base64


# 이미지를 Base64로 인코딩
def encode_image_to_base64(image):    
    _, buffer = cv2.imencode('.jpg', image)
    return base64.b64encode(buffer).decode('utf-8')

def analyze_emotion(face_image):
    # 이미지 Base64 인코딩
    image_base64 = encode_image_to_base64(face_image)

    response = client.chat.completions.create(
      model = "gpt-4o-mini",     
      messages = [
        {
          "role": "user",
          "content": [
            {"type": "text", "text": "이미지의 표정을 보고 기쁘면 [Happy],슬프면 [Sad],놀란 표정이면 [Surprised], 화났으면 [Angry],무표정은 [Neutral]으로 말해줘"},
            {
              "type": "image_url",
              "image_url": {
                 "url": f"data:image/jpeg;base64,{image_base64}"
              },
            },
          ],
        }
      ],
      
    )    
    return response.choices[0].message.content
   

# OpenCV 얼굴 감지기 로드
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# === 쓰레드를 이용한 프레임 버퍼 구현 ===
class Camera:
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        self.frame = None
        self.emotion = "[Neutral]"
        self.count = 0
        self.lock = threading.Lock()
        self.running = True
        thread = threading.Thread(target=self.update, daemon=True)
        thread.start()

    def update(self):
        while self.running:
            self.count += 1
            success, frame = self.cap.read()
            # print(self.count)
            
            if success:                
                # 여기서 원하는 이미지 처리 적용  
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                
                for (x, y, w, h) in faces:
                    face_roi = frame[y:y+h, x:x+w]
                    
                    if self.count > 60 :   # 60 frame당 1번 만 OpenAI 호출
                        self.count = 0      # reset
                        self.emotion = analyze_emotion(face_roi)
                        print(self.emotion)
                    # print(self.count)
                
                    # rect_img = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    # text_img  = cv2.putText(rect_img, self.emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)        
                    # print(self.emotion)    
                    
                with self.lock:
                    self.frame = frame

    def get_frame(self):
        with self.lock:
            if self.frame is None:
                return None
            return self.frame.copy()

    def stop(self):
        self.running = False
        self.cap.release()

camera = Camera()  # 전역 카메라 객체 생성

           
# Flask 프레임 스트리밍
def generate_frames():
    while True:
        frame = camera.get_frame()
        if frame is None:
            continue

        _, buffer = cv2.imencode('.jpg', frame)
        frame_bytes = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    try:
        app.run(host='0.0.0.0', port=5000, debug=False, threaded=True)
    finally:
        camera.stop()  # 종료 시 카메라 정리

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.23:5000
Press CTRL+C to quit


[Neutral]


192.168.0.23 - - [13/Mar/2025 16:43:07] "GET / HTTP/1.1" 200 -
192.168.0.23 - - [13/Mar/2025 16:43:07] "GET /video_feed HTTP/1.1" 200 -


[Neutral]
[Happy]
[Surprised]
[Neutral]
[Neutral]
[Neutral]
[Neutral]
[Angry]
[Neutral]
[Neutral]
[Neutral]
[Neutral]
[Neutral]


### [4] 속도 개선 - 2 : WebSocket을 사용하여 속도 개선
HTTP 스트리밍 방식 (multipart/x-mixed-replace)은 프레임을 전송하는 과정에서 많은 오버헤드가 발생한다.
따라서 WebSocket을 사용하여 프레임을 직접 전송하면 속도를 대폭 개선할 수 있다.

In [1]:
from dotenv import load_dotenv
import os

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")
# print(API_KEY)

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

from flask import Flask, request, jsonify, render_template
app = Flask(__name__)

In [2]:
# ! pip install flask-socketio

In [1]:
from dotenv import load_dotenv
import os

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")
# print(API_KEY)

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

from flask import Flask, request, jsonify, render_template
app = Flask(__name__)

In [2]:
from flask_socketio import SocketIO
import cv2
import base64
import threading
import numpy as np

socketio = SocketIO(app, cors_allowed_origins="*")
# 이미지를 Base64로 인코딩
def encode_image_to_base64(image):    
    _, buffer = cv2.imencode('.jpg', image)
    return base64.b64encode(buffer).decode('utf-8')

def analyze_emotion(face_image):
    # 이미지 Base64 인코딩
    image_base64 = encode_image_to_base64(face_image)

    response = client.chat.completions.create(
      model = "gpt-4o-mini",     
      messages = [
        {
          "role": "user",
          "content": [
            {"type": "text", "text": "이미지의 표정을 보고 기쁘면 [Happy],슬프면 [Sad],놀란 표정이면 [Surprised], 화났으면 [Angry],무표정은 [Neutral]으로 말해줘"},
            {
              "type": "image_url",
              "image_url": {
                 "url": f"data:image/jpeg;base64,{image_base64}"
              },
            },
          ],
        }
      ],
      
    )    
    return response.choices[0].message.content
   

# OpenCV 얼굴 감지기 로드
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


class Camera:
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        self.frame = None
        self.emotion = "[Neutral]"
        self.count = 0        
        self.lock = threading.Lock()
        self.running = True
        thread = threading.Thread(target=self.update, daemon=True)
        thread.start()

    def update(self):
        while self.running:
            self.count += 1
            success, frame = self.cap.read()
            # print(self.count)
            
            if success:                
                # 여기서 원하는 이미지 처리 적용  
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                
                for (x, y, w, h) in faces:
                    face_roi = frame[y:y+h, x:x+w]
                    
                    if self.count > 50 :   # 50 frame당 1번 만 OpenAI 호출
                        self.count = 0     # reset
                        self.emotion = analyze_emotion(face_roi)
                        # print(self.emotion)
                        
                        # 감정 데이터를 전송
                        socketio.emit("emotion_update", {"emotion": self.emotion})        
                        
                    # print(self.count)
                
                    frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    # frame  = cv2.putText(frame, self.emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)        
                    # print(self.emotion)    
                    
                with self.lock:
                    self.frame = frame

    def get_frame(self):
        with self.lock:
            if self.frame is None:
                return None
            return self.frame.copy()

    def stop(self):
        self.running = False
        self.cap.release()

camera = Camera()

@app.route('/')
def index():
    return render_template('index_ws.html')  # WebSocket 버전 HTML

@socketio.on('request_frame')
def send_frame():
    frame = camera.get_frame()
    if frame is not None:
        _, buffer = cv2.imencode('.jpg', frame)
        encoded_frame = base64.b64encode(buffer).decode('utf-8')
        socketio.emit('frame', encoded_frame)
       
if __name__ == '__main__':
    try:
        socketio.run(app, host='0.0.0.0', port=5000, debug=False,allow_unsafe_werkzeug=True)
    finally:
        camera.stop()

Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.23:5000
Press CTRL+C to quit
192.168.0.23 - - [13/Mar/2025 18:31:45] "GET /socket.io/?EIO=4&transport=polling&t=PMEs6_D HTTP/1.1" 200 -
192.168.0.23 - - [13/Mar/2025 18:31:45] "POST /socket.io/?EIO=4&transport=polling&t=PMEs7K5&sid=AFr-jY94hIvD4AdwAAAA HTTP/1.1" 200 -
192.168.0.23 - - [13/Mar/2025 18:31:45] "GET /socket.io/?EIO=4&transport=polling&t=PMEs7K6&sid=AFr-jY94hIvD4AdwAAAA HTTP/1.1" 200 -
192.168.0.23 - - [13/Mar/2025 18:31:53] "GET /socket.io/?EIO=4&transport=websocket&sid=AFr-jY94hIvD4AdwAAAA HTTP/1.1" 200 -
192.168.0.23 - - [13/Mar/2025 18:31:53] "GET / HTTP/1.1" 200 -
192.168.0.23 - - [13/Mar/2025 18:31:53] "GET /socket.io/?EIO=4&transport=polling&t=PMEs9LN HTTP/1.1" 200 -
192.168.0.23 - - [13/Mar/2025 18:31:53] "POST /socket.io/?EIO=4&transport=polling&t=PMEs9LV&sid=nICSkNCunrud7wziAAAC HTTP/1.1" 200 -
192.168.0.23 - - [13/Mar/2025 18:31:53] "GET /socket.io/?EIO=4&tran